In [1]:
!pip install SpeechRecognition
!pip install nltk
!pip install langdetect

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [25]:
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
import re

In [26]:
df = pd.read_csv('mental_health.csv')
df.head()

,text,label
0,dear american teens question dutch person hear...,0
1,nothing look forward lifei dont many reasons k...,1
2,music recommendations im looking expand playli...,0
3,im done trying feel betterthe reason im still ...,1
4,worried year old girl subject domestic physic...,1


In [27]:
df.tail()

,text,label
27972,posting everyday people stop caring religion ...,0
27973,okay definetly need hear guys opinion ive pret...,0
27974,cant get dog think ill kill myselfthe last thi...,1
27975,whats point princess bridei really think like ...,1
27976,got nudes person might might know snapchat do ...,0


In [28]:
df.shape

(27977, 2)

In [29]:
#Preprocessing
df.isna().sum()

text     0
label    0
dtype: int64

In [30]:
df.duplicated().sum()

5

In [31]:
df = df.drop_duplicates()

In [32]:
df['label'].unique()

array([0, 1])

In [33]:
df['label'].value_counts()

label
0    14134
1    13838
Name: count, dtype: int64

In [34]:
df['Total Words'] = df['text'].apply(lambda x: len(x.split()))

In [35]:
def count_total_words(text):
    char = 0
    for word in text.split():
        char += len(word)
    return char

df['Total Characters'] = df["text"].apply(count_total_words)

In [36]:

df.head()

,text,label,Total Words,Total Characters
0,dear american teens question dutch person hear...,0,23,112
1,nothing look forward lifei dont many reasons k...,1,20,98
2,music recommendations im looking expand playli...,0,64,387
3,im done trying feel betterthe reason im still ...,1,100,493
4,worried year old girl subject domestic physic...,1,311,1819


In [37]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/u116710/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/u116710/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/u116710/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [38]:
def preprocess_text(text):
    # remove URLs
    text = re.sub(r'http\S+', '', text)

    # convert text to lowercase
    text = text.lower()

    # remove numbers and punctuation
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)

    # tokenize text into words
    words = word_tokenize(text)

    # remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # stemmed words
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]

    # lemmatize words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    # join words back into text
    text = ' '.join(words)

    # tokenize text into sentences
    sentences = sent_tokenize(text)

    return " ".join(sentences)

In [39]:
df['text'] = df['text'].apply(preprocess_text)

In [40]:
df.head()

,text,label,Total Words,Total Characters
0,dear american teen question dutch person heard...,0,23,112
1,noth look forward lifei dont mani reason keep ...,1,20,98
2,music recommend im look expand playlist usual ...,0,64,387
3,im done tri feel betterth reason im still aliv...,1,100,493
4,worri year old girl subject domest physicalmen...,1,311,1819


In [41]:
df['Total Words After Transformation'] = df['text'].apply(lambda x: len(x.split()))

In [42]:
df.head()

,text,label,Total Words,Total Characters,Total Words After Transformation
0,dear american teen question dutch person heard...,0,23,112,23
1,noth look forward lifei dont mani reason keep ...,1,20,98,19
2,music recommend im look expand playlist usual ...,0,64,387,61
3,im done tri feel betterth reason im still aliv...,1,100,493,97
4,worri year old girl subject domest physicalmen...,1,311,1819,296


In [43]:
from sklearnex import patch_sklearn
patch_sklearn()

from sklearn.model_selection import train_test_split
X = df["text"]
y = df['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 42, stratify = y)

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [45]:
from sklearnex import patch_sklearn
patch_sklearn()

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection, svm

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [46]:
tfidf = TfidfVectorizer(max_features= 2500, min_df= 2)
X_train = tfidf.fit_transform(X_train).toarray()
X_test = tfidf.transform(X_test).toarray()

In [47]:
classifiers = [
    ('LogisticRegression', LogisticRegression()),
    ('SVC', SVC()),
    ('DecisionTreeClassifier', DecisionTreeClassifier()),
    ('RandomForestClassifier', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('MultinomialNB', MultinomialNB())
]

In [48]:
# Train and evaluate each classifier
import pickle

for name, classifier in classifiers:
    print(f"Training {name}...")
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {accuracy:.4f}")

    # Save the trained model as a pickle file
    model_filename = f"{name}_model(audio).pkl"
    with open(model_filename, 'wb') as file:
        pickle.dump((classifier,tfidf), file)
    print(f"Saved {name} model as {model_filename}")

    # Print predicted labels alongside true labels
    predictions = pd.DataFrame({'True_Labels': y_test, 'Predicted_Labels': y_pred})
    print(f"Predictions using {name} model:")
    print(predictions)

Training LogisticRegression...
LogisticRegression Accuracy: 0.9137
Saved LogisticRegression model as LogisticRegression_model(audio).pkl
Predictions using LogisticRegression model:
      True_Labels  Predicted_Labels
0               1                 1
1               1                 1
2               0                 1
3               1                 1
4               0                 0
...           ...               ...
5590            0                 0
5591            0                 0
5592            0                 0
5593            1                 0
5594            0                 0

[5595 rows x 2 columns]
Training SVC...
SVC Accuracy: 0.9172
Saved SVC model as SVC_model(audio).pkl
Predictions using SVC model:
      True_Labels  Predicted_Labels
0               1                 1
1               1                 1
2               0                 1
3               1                 1
4               0                 0
...           ...               ...
5590

In [49]:
import pickle
import numpy as np
from flask import Flask, render_template, request

from sklearn.feature_extraction.text import TfidfVectorizer

# Load the trained SVC model and fitted TF-IDF vectorizer from the pickle file
with open('models/SVC_model(audio).pkl', 'rb') as model_file:
    model, fitted_tfidf = pickle.load(model_file)

# Sample text input
sample_text_input = "I never realized that exercise can help you feel so good make you happy and motivated both physically and emotionally"

# Preprocess the text input using the fitted TF-IDF vectorizer
preprocessed_input = fitted_tfidf.transform([sample_text_input]).toarray()

# Predict label
predicted_label = model.predict(preprocessed_input)

# Print the predicted label
print("Predicted Label:", predicted_label)

Predicted Label: [0]
